In [ ]:
import pandas as pd
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
df = pd.read_csv('news_cleaned.csv', sep = ";")

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
def split_into_sentences(s):
    # Remove the leading and trailing square brackets and single quotes
    s = s[2:-2]

    # Split the string into a list of sentences
    sentences = s.split("', '")

    return sentences

# Apply the function to the 'sentences' column
df['sentences'] = df['sentences'].apply(split_into_sentences)

In [ ]:
AAPL_df = df[df['ticker'] == 'AAPL']
samples_100_AAPL = AAPL_df.sample(100)

In [ ]:
import torch

In [ ]:
def classify_sentences(sentences):
    # Initialize the lists of headlines and scores
    predictions = []
    positives = []
    negatives = []
    neutrals = []

    # Process each sentence
    for sentence in sentences:
        # Tokenize the sentence
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

        # Get the model's prediction
        outputs = model(**inputs)

        # Get the predicted scores
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = outputs.logits.argmax(dim=-1).item()

        # Add the sentence and scores to the lists
        predictions.append(prediction)
        positives.append(scores[:, 0].item())
        negatives.append(scores[:, 1].item())
        neutrals.append(scores[:, 2].item())

    return predictions, positives, negatives, neutrals

# Apply the function to the 'sentences' column and concatenate the results
samples_100_AAPL[['Predictions', 'Positive', 'Negative', 'Neutral']] = samples_100_AAPL['sentences'].apply(lambda x: pd.Series(classify_sentences(x)))

In [ ]:
samples_100_AAPL

In [ ]:
samples_100_AAPL.to_csv('output.csv', index=True)